## ML regression to predict the Efficacy of an active G9a inhibitor. Dataset 2 (reduced with the water solubility dataset and with addition of the solubility feature)

### Content   <a name="content"></a>

1. [Load data](#1)
2. [Regression Machine Learning](#2)
3. [Cros-validation](#3)
4. [Calculate the relative error of the Gradient Boosting Regressor model](#4)
5. [Feature importance of the Gradient Boosting Regressor model](#5)
6. [Comparison of the first six features from the feature importance results](#6)
7. [Hyperparameter tuning of the model with the reduced features](#7)
8. [Relative error of the reduced data model](#8)

## Load data<a name="1"></a>

In [1]:
# pip install modin[ray] 
# pip install sidetable

In [2]:
import pandas as pd 

# # loading the dataset for the regression ML
df = pd.read_csv('data_reg_with_water_solub.csv', index_col=[0])
# Avoid some columns to be truncated during df display
pd.set_option('display.max_columns', None)
# Display the data frame
print('Shape of df: ', df.shape)
df.head()

Shape of df:  (3890, 66)


,index,CID,SID,Efficacy,SMILES,MW,MF,TPSA,XL,HAC,HBDC,HBAC,RBC,CBUC,MMX6,MMX,SX6,SX,MMY6,MMY,SY6,SY,Volume_1,Volume_2,MMX6_3D,MMX_3D,SX6_3D,SX_3D,MMY6_3D,MMY_3D,SY6_3D,SY_3D,MMZ6_3D,MMZ_3D,SZ6_3D,SZ_3D,Volume_1_3D,XY_3D_volume,XZ_3D_volume,YZ_3D_volume,C_relative,H_relative,O_relative,S_relative,N_relative,Br_relative,Cl_relative,F_relative,C,H,O,S,N,Br,Cl,F,C_rel_2D,allAtoms_rel_2D,C_rel_XY_3D,allAtoms_rel_XY_3D,C_rel_XZ_3D,allAtoms_rel_XZ_3D,C_rel_YZ_3D,allAtoms_rel_YZ_3D,Similarity,Solubility_at_pH_7_4
0,0,135856003,26729454,72.9655,CC1C(=O)N/C(=N\N=C\C2=CC(=CC=C2)[N+](=O)[O-])/S1,278.29,C11H10N4O3S,125.0,2.2,19,1,6,2,1,6.6342,8.9727,0.089578,-0.356796,10.0201,10.0201,-0.487187,-0.275276,10.703674,3.062773e-01,10.4533,13.4223,0.485431,0.053612,2.9936,5.1742,0.513676,0.355558,1.1650,3.0878,-2.424383,-1.454583,154.967504,2.341853,0.810366,0.061954,0.38,0.34,0.10,0.03,0.14,0.0,0.0,0.0,47.48,3.62,17.25,11.52,20.13,0.0,0.0,0.0,0.662089,0.895470,3.491883,2.594082,8.972790,4.346881,2.569614,1.675691,0.065,0.1
1,1,135845987,26660913,147.3170,CC(=O)NC1=C(N=C(S1)N/N=C\C2=CC=C(C=C2)O)C3=CC=CS3,358.40,C16H14N4O2S2,143.0,3.4,24,3,7,5,1,7.0834,7.2974,-0.070702,0.028400,12.5835,12.5835,0.114455,0.268933,0.000006,2.241061e-07,11.7024,13.8931,0.065879,0.130764,6.9136,8.9136,-0.930758,-0.466980,2.4349,4.3407,-0.210250,-0.047959,69.723704,0.040203,0.078547,0.012436,0.42,0.37,0.05,0.05,0.11,0.0,0.0,0.0,53.62,3.94,8.93,17.89,15.63,0.0,0.0,0.0,0.562912,0.579918,1.692664,1.558641,4.806111,3.200659,2.839377,2.053494,0.054,13.7
2,2,135829861,49679361,174.8850,C1=CC2=C(C=CC(=C2N=NC3=CC=C(C=C3)C(=O)O)O)N=C1,293.28,C16H11N3O3,95.1,3.1,22,2,6,3,1,6.4686,6.4686,0.643863,0.698222,11.2534,11.8613,0.319010,0.365890,0.003303,5.384583e-06,10.1926,12.9043,0.323032,0.324770,4.9932,7.6809,0.171935,-0.007312,0.6980,1.1042,-0.315743,-0.253670,8.764451,1.172396,0.104192,0.209574,0.48,0.33,0.09,0.00,0.09,0.0,0.0,0.0,65.53,3.78,16.37,0.00,14.33,0.0,0.0,0.0,0.574813,0.545353,2.041296,1.680051,14.602579,11.686560,7.153582,6.956077,0.050,24.7
3,3,135829849,49673210,189.2240,CCOC1=CC=C(C=C1)NC(=O)/C(=C(\C)/O)/C=NC2=CC=CC=C2,324.40,C19H20N2O3,70.9,3.8,24,2,4,6,1,15.3302,15.3302,-0.063507,-0.037934,7.2036,7.2036,-0.223659,-0.149767,0.000000,3.739519e-09,12.6857,14.2391,-0.324084,-0.305440,7.6102,8.9715,-0.653389,-0.602524,2.7585,4.4880,0.219574,0.105079,13.375666,0.014993,0.059015,0.040756,0.43,0.45,0.07,0.00,0.05,0.0,0.0,0.0,70.35,6.21,14.80,0.00,8.64,0.0,0.0,0.0,2.128130,2.128130,1.666934,1.587148,4.598767,3.172705,2.758818,1.998997,0.059,1.0
4,4,135825093,26658783,166.8680,CCOC1=C(C=CC(=C1)/C=N\NC2=NC(=C(S2)NC(=O)C)C3=...,410.50,C21H22N4O3S,124.0,4.4,29,3,7,7,1,16.8859,16.8859,0.085001,-0.000425,9.6230,11.9205,0.105757,0.021858,9.625909,5.822748e-02,13.1863,15.5571,-0.004718,0.086889,7.4717,9.5477,1.094927,0.563269,3.4022,5.2265,-0.578542,-0.502618,80.618934,0.802356,0.111462,0.517832,0.41,0.43,0.06,0.02,0.08,0.0,0.0,0.0,61.45,5.40,11.69,7.81,13.65,0.0,0.0,0.0,1.754744,1.416543,1.764833,1.629408,3.875816,2.976581,2.196138,1.826787,0.054,9.7


In [3]:
# Check for NaN
df.isnull().values.any()

False

In [4]:
df.describe(include="all")

,index,CID,SID,Efficacy,SMILES,MW,MF,TPSA,XL,HAC,HBDC,HBAC,RBC,CBUC,MMX6,MMX,SX6,SX,MMY6,MMY,SY6,SY,Volume_1,Volume_2,MMX6_3D,MMX_3D,SX6_3D,SX_3D,MMY6_3D,MMY_3D,SY6_3D,SY_3D,MMZ6_3D,MMZ_3D,SZ6_3D,SZ_3D,Volume_1_3D,XY_3D_volume,XZ_3D_volume,YZ_3D_volume,C_relative,H_relative,O_relative,S_relative,N_relative,Br_relative,Cl_relative,F_relative,C,H,O,S,N,Br,Cl,F,C_rel_2D,allAtoms_rel_2D,C_rel_XY_3D,allAtoms_rel_XY_3D,C_rel_XZ_3D,allAtoms_rel_XZ_3D,C_rel_YZ_3D,allAtoms_rel_YZ_3D,Similarity,Solubility_at_pH_7_4
count,3890.000000,3.890000e+03,3.890000e+03,3890.000000,3890,3890.000000,3890,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.0,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3.890000e+03,3.890000e+03,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3.890000e+03,3890.000000,3.890000e+03,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000,3890.000000
unique,NaN,NaN,NaN,NaN,3890,NaN,2838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,CC1C(=O)N/C(=N\N=C\C2=CC(=CC=C2)[N+](=O)[O-])/S1,NaN,C18H17N3O2S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,1,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,1944.500000,1.134585e+07,3.611781e+07,132.911167,NaN,299.861143,NaN,79.775416,2.744679,20.894859,1.320566,4.268123,3.651414,1.0,9.596785,10.286804,0.025353,0.023352,8.158382,8.763759,-0.052072,-0.024514,8.851506e+00,5.244091e-02,10.221921,12.306994,0.008828,0.017040,4.622067,6.681055,0.003001,0.000904,2.297309,4.037785,-0.006760,-0.008338,6.632588e+01,0.484506,8.630829e-01,0.762899,0.414787,0.416226,0.059111,0.018355,0.080378,0.001190,0.005180,0.004141,60.254805,5.194190,11.088846,6.577746,13.248085,0.881794,1.905689,0.848920,1.405983,1.341871,2.468800,1.934973,157.000569,59.782415,69.179706,33.285931,0.055878,24.361632
std,1123.090602,3.013587e+07,1.242266e+07,29.294427,NaN,46.148193,NaN,31.013529,1.136054,3.446987,0.910392,1.540684,1.733410,0.0,2.911908,2.751989,0.303721,0.268670,2.887211,2.710905,0.365724,0.298293,5.212942e+01,2.911455e-01,2.442560,2.368003,0.324426,0.259901,1.468863,1.403279,0.442323,0.292514,1.103894,1.322431,0.585349,0.407594,8.487196e+01,1.678814,1.143204e+01,11.348781,0.045202,0.067071,0.042085,0.021932,0.040790,0.006509,0.014802,0.016681,9.460849,1.503441,7.417339,7.322493,6.120085,4.670897,5.122398,3.182549,0.930720,0.696470,1.112410,0.600815,981.269775,571.847622,468.845781,339.451730,0.015490,17.227432
min,0.000000,4.757000e+03,8.426290e+05,62.000400,NaN,97.070000,NaN,3.200000,-3.700000,7.000000,0.000000,1.000000,0.000000,1.0,1.963500,2.605200,-1.212065,-0.877719,0.750000,1.950000,-1.308379,-1.071139,0.000000e+00,2.260095e-38,2.145400,4.589300,-1.465220,-0.917542,0.394700,3.024900,-2.090206,-1.113708,0.000300,0.000500,-3.102060,-5.387489,3.807599e-07,0.000129,2.000274e-07,0.000007,0.170000,0.140000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.830000,0.890000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.158514,0.158514,0.576067,0.798337,1.002917,0.908398,0.26

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3890 entries, 0 to 3889
Data columns (total 66 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 3890 non-null   int64  
 1   CID                   3890 non-null   int64  
 2   SID                   3890 non-null   int64  
 3   Efficacy              3890 non-null   float64
 4   SMILES                3890 non-null   object 
 5   MW                    3890 non-null   float64
 6   MF                    3890 non-null   object 
 7   TPSA                  3890 non-null   float64
 8   XL                    3890 non-null   float64
 9   HAC                   3890 non-null   int64  
 10  HBDC                  3890 non-null   int64  
 11  HBAC                  3890 non-null   int64  
 12  RBC                   3890 non-null   int64  
 13  CBUC                  3890 non-null   int64  
 14  MMX6                  3890 non-null   float64
 15  MMX                   3890

[<a href="#content">Back to top</a>]

## Regression Machine Learning <a name="2"></a>

In [6]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Separate the training columns from the target column
X = df.drop(columns=['CID', 'SID', 'SMILES','MF', 'Efficacy']) 
y = df['Efficacy'] 

# Split the data set into train and test parts 
X_train_unscaled, X_test_unscaled, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20,
                                                    random_state=5) 
# # Standardise the data points
sc = StandardScaler()
X_train = sc.fit_transform(X_train_unscaled)
X_test = sc.transform(X_test_unscaled)

# Print the shape of each part
print("Shapes:")
print("X_train: ", X_train.shape)
print("X_test:  ", X_test.shape)
print("y_train: ", y_train.shape)
print("y_test:  ", y_test.shape)

Shapes:
X_train:  (3112, 61)
X_test:   (778, 61)
y_train:  (3112,)
y_test:   (778,)


In [7]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Instantiate the algorithms that will be used, placing them in a dictionary 
regs = {"SVR":SVR(kernel='linear'),
        "DecisionTree":DecisionTreeRegressor(), 
        "RandomForest":RandomForestRegressor(), 
        "GradientBoost":GradientBoostingRegressor(),}

In [8]:
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Create statistics with the results of training with different algorithms
def model_fit(regs):
    fitted_model={}
    model_result = pd.DataFrame()
    for model_name, model in regs.items():
        model.fit(X_train,y_train)
        fitted_model.update({model_name:model})
        model_dict = {}
        model_dict['Algorithm'] = model_name
        model_dict['RMSE_Train'] = round(root_mean_squared_error(y_train, model.predict(X_train)),2)
        model_dict['RMSE_Test'] = round(root_mean_squared_error(y_test, model.predict(X_test)),2)
        model_dict['MAE_Train'] = round(mean_absolute_error(y_train, model.predict(X_train)),2)
        model_dict['MAE_Test'] = round(mean_absolute_error(y_test, model.predict(X_test)),2)
        model_dict['R2_Train'] = round(r2_score(y_train, model.predict(X_train)),2)
        model_dict['R2_Test'] = round(r2_score(y_test, model.predict(X_test)),2)
        model_result = model_result._append(model_dict,ignore_index=True)
    return fitted_model, model_result

fitted_model, model_result = model_fit(regs)
model_result.sort_values(by=['MAE_Test'],ascending=True)

,Algorithm,RMSE_Train,RMSE_Test,MAE_Train,MAE_Test,R2_Train,R2_Test
2,RandomForest,10.71,28.96,8.33,22.50,0.87,0.04
3,GradientBoost,24.55,29.18,19.37,22.71,0.29,0.02
0,SVR,28.54,29.78,22.28,23.13,0.05,-0.02
1,DecisionTree,0.00,42.02,0.00,33.09,1.00,-1.03


[<a href="#content">Back to top</a>]

## Cross-validation <a name="3"></a>

In [9]:
from sklearn.model_selection import cross_val_score
import numpy as np

# Create statistics with the results of cross-validation
def model_CV(regs):
    fitted_model={}
    model_cv_result = pd.DataFrame()
    for model_name, model in regs.items():
        fitted_model.update({model_name:model})
        scores = cross_val_score(model, X_train, y_train, cv=5,
                        scoring=('neg_mean_absolute_error'))
        scores = -scores
        model_dict = {}
        model_dict['Algorithm'] = model_name
        model_dict['CV_MAE'] = round(np.mean(scores), 2)
        model_dict['Sta Dev MAE'] = round(np.std(scores), 2)
        model_dict['List of MAE'] = np.round(scores, 2)
        model_cv_result = model_cv_result._append(model_dict,ignore_index=True)
    return fitted_model, model_cv_result

fitted_model, model_cv_result = model_CV(regs)
model_cv_result.sort_values(by=['CV_MAE'],ascending= True)

,Algorithm,CV_MAE,Sta Dev MAE,List of MAE
2,RandomForest,22.43,0.12,"[22.54, 22.23, 22.53, 22.35, 22.5]"
3,GradientBoost,22.47,0.30,"[22.74, 22.0, 22.75, 22.23, 22.63]"
0,SVR,22.77,0.33,"[23.34, 22.37, 22.93, 22.64, 22.57]"
1,DecisionTree,31.78,1.23,"[32.18, 33.15, 29.92, 32.84, 30.82]"


[<a href="#content">Back to top</a>]

## Calculate the relative error of the Gradient Boosting Regressor model  <a name="4"></a>

In [13]:
import sklearn.metrics as metrics
from sklearn.metrics import r2_score

# Instantiate and train a model
model = SVR().fit(X_train, y_train)

# Predict 
pred = model.predict(X_test)

# Evaluate
print('Mean Absolute Error (MAE):', round(metrics.mean_absolute_error(y_test, pred),2))
print('Mean Squared Error (MSE):', round(metrics.mean_squared_error(y_test, pred),2))
print('Root Mean Squared Error (RMSE):', round(np.sqrt(metrics.mean_squared_error(y_test, pred))))
print("R2 score for perfect model is:", round(r2_score(y_test, pred), 2))

Mean Absolute Error (MAE): 22.99
Mean Squared Error (MSE): 869.86
Root Mean Squared Error (RMSE): 29
R2 score for perfect model is: 0.0


In [14]:
# Create a data frame with the test values 
data_verify=pd.DataFrame(y_test.tolist(),columns=["Real Values"])

# Create a data frame with the values predicted 
data_predicted=pd.DataFrame(pred.tolist(),columns=["Predicted Values"])

# Concatenate the data frames with the test and the values predicted
final_output=pd.concat([data_verify,data_predicted],axis=1)

# Create column with the difference between the test and prediction values
final_output["Difference"]= np.abs(final_output["Real Values"]-final_output["Predicted Values"])
final_output["Relative proportion Difference/Real Value"]= (final_output["Difference"]/final_output["Real Values"])

# Display the resulted data frame 
final_output

,Real Values,Predicted Values,Difference,Relative proportion Difference/Real Value
0,173.407,135.400821,38.006179,0.219173
1,162.565,130.050047,32.514953,0.200012
2,124.315,137.127960,12.812960,0.103068
3,152.531,129.428230,23.102770,0.151463
4,125.828,134.458691,8.630691,0.068591
...,...,...,...,...
773,147.772,131.716256,16.055744,0.108652
774,134.144,135.089689,0.945689,0.007050
775,123.336,129.648830,6.312830,0.051184
776,139.113,138.021029,1.091971,0.007850


In [15]:
# Mean of the relative error
df_reg_rel_mean = final_output["Relative proportion Difference/Real Value"].mean()
print("Relative error: ", df_reg_rel_mean)

Relative error:  0.18118513114771656


[<a href="#content">Back to top</a>]